In [2]:
from sklearn.cluster import DBSCAN
import numpy as np

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold
# import numpy as np
import pandas as pd

In [4]:
from create_matrix import *

from tensor_custom_core import *
from create_matrix import *
from tensor_custom_core import *
from degree_days import dds
appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}

APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
region = "SanDiego"
year = 2014

import os
from degree_days import dds
import autograd.numpy as np




def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

In [5]:
def get_tensor(df, dfc):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

    static_cols = ['area', 'total_occupants', 'num_rooms']
    static_df = df[static_cols]
    static_df = static_df.div(static_df.max())
    weather_values = np.array(dds[2014][region][start - 1:stop - 1]).reshape(-1, 1)

    dfc = df.copy()

    df = dfc[energy_cols]
    col_max = df.max().max()
    col_min = df.min().min()
    # df = (1.0 * (df - col_min)) / (col_max - col_min)
    tensor = df.values.reshape((len(df), 7, stop - start))
    M, N, O = tensor.shape
    return tensor

In [6]:
au_df, au_dfc = create_matrix_single_region("Austin", year)
au_tensor = get_tensor(au_df, au_dfc)
static_au = au_df[['area','total_occupants','num_rooms']].copy()
static_au['area'] = static_au['area'].div(4000)
static_au['total_occupants'] = static_au['total_occupants'].div(8)
static_au['num_rooms'] = static_au['num_rooms'].div(8)
static_au = static_au.values

In [7]:
sd_df, sd_dfc = create_matrix_single_region("SanDiego", year)
sd_tensor = get_tensor(sd_df, sd_dfc)
static_sd = sd_df[['area','total_occupants','num_rooms']].copy()
static_sd['area'] = static_sd['area'].div(4000)
static_sd['total_occupants'] = static_sd['total_occupants'].div(8)
static_sd['num_rooms'] = static_sd['num_rooms'].div(8)
static_sd = static_sd.values

In [16]:
sd_agg = sd_df.loc[:, 'aggregate_1':'aggregate_12']
au_agg = au_df.loc[:, 'aggregate_1':'aggregate_12']

In [19]:
sd_agg = np.nan_to_num(sd_agg)
au_agg = np.nan_to_num(au_agg)

In [35]:
def distance(x, y):
    return np.linalg.norm(x - y)

In [96]:
db = DBSCAN(eps=500, min_samples=1, metric=distance).fit(sd_agg)
sd_labels = db.labels_
db = DBSCAN(eps=500, min_samples=1, metric=distance).fit(au_agg)
au_labels = db.labels_


In [97]:
print len(set(au_labels)), len(set(sd_labels))

136 10


In [98]:
len(au_agg)
n_sample_sd = len(sd_agg)
n_sample_au = len(au_agg)

In [99]:
W_sd = np.zeros((n_sample_sd, n_sample_sd))
W_au = np.zeros((n_sample_au, n_sample_au))

In [101]:
for i in range(n_sample_sd):
    label = sd_labels[i]
    neighbours = [j for j,x in enumerate(sd_labels) if x == label]
    for k in neighbours:
        W_sd[i][k] = 1
        
for i in range(n_sample_au):
    label = au_labels[i]
    neighbours = [j for j,x in enumerate(au_labels) if x == label]
    for k in neighbours:
        W_au[i][k] = 1
        

In [109]:
D_sd = np.zeros((n_sample_sd, n_sample_sd))
D_au = np.zeros((n_sample_au, n_sample_au))
for i in range(n_sample_sd):
    D_sd[i][i] = sum(W_sd[i])
for i in range(n_sample_au):
    D_au[i][i] = sum(W_au[i])

In [110]:
L_sd = D_sd - W_sd
L_au = D_au - W_au

In [ ]:
def cost_graph_laplacian(H, A, T, F, L, E_np_masked, K, case):
    HAT = multiply_case(H, A, T, case)
    mask = ~np.isnan(E_np_masked)
    error_1 = (HAT - E_np_masked)[mask].flatten()
    
    HF = np.einsum('na, ab->nb', H, F)
    mask_static = ~np.isnan(k)
    error_2 = (HF - k)[mask_static].flatten()
    
    FL = np.einsum('ab, ')
    
    return np.sqrt((error_1**2).mean()) + np.sqrt((error_2**2).mean()) + error_3